In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime
import uuid

class Person:
    def __init__(self, name, address, phone):
        self.name = name
        self.address = address
        self.phone = phone
    
    def get_details(self):
        return f"Name: {self.name}, Address: {self.address}, Phone: {self.phone}"

class Account(ABC):
    def __init__(self, owner, initial_balance=0.0):
        self.account_number = str(uuid.uuid4())[:8]  # Generate a unique 8-char ID
        self.balance = initial_balance
        self.owner = owner
        self.transactions = []
    
    @abstractmethod
    def withdraw(self, amount):
        pass
    
    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            self.transactions.append(Transaction("Deposit", amount))
        else:
            raise ValueError("Deposit amount must be greater than zero")
    
    def get_balance(self):
        return self.balance
    
    def show_account_details(self):
        return f"Account No: {self.account_number}, Balance: {self.balance}, Owner: {self.owner.name}"

class SavingsAccount(Account):
    def __init__(self, owner, initial_balance=0.0, interest_rate=0.03):
        super().__init__(owner, initial_balance)
        self.interest_rate = interest_rate
    
    def apply_interest(self):
        interest = self.balance * self.interest_rate
        self.balance += interest
        self.transactions.append(Transaction("Interest Applied", interest))
    
    def withdraw(self, amount):
        if amount > 0 and amount <= self.balance:
            self.balance -= amount
            self.transactions.append(Transaction("Withdrawal", amount))
        else:
            raise ValueError("Insufficient balance or invalid amount")

class BusinessAccount(Account):
    def __init__(self, owner, initial_balance=0.0, overdraft_limit=1000.0):
        super().__init__(owner, initial_balance)
        self.overdraft_limit = overdraft_limit
    
    def withdraw(self, amount):
        if amount > 0 and self.balance - amount >= -self.overdraft_limit:
            self.balance -= amount
            self.transactions.append(Transaction("Withdrawal", amount))
        else:
            raise ValueError("Withdrawal exceeds overdraft limit or invalid amount")

class Customer(Person):
    def __init__(self, name, address, phone):
        super().__init__(name, address, phone)
        self.accounts = []
    
    def add_account(self, account):
        self.accounts.append(account)
    
    def remove_account(self, account_number):
        self.accounts = [acc for acc in self.accounts if acc.account_number != account_number]
    
    def get_accounts_summary(self):
        return [acc.show_account_details() for acc in self.accounts]

class Admin(Person):
    def __init__(self, name, address, phone, employee_id):
        super().__init__(name, address, phone)
        self.employee_id = employee_id
    
    def approve_account(self, customer, account):
        customer.add_account(account)
    
    def view_all_customers(self, bank):
        return [customer.get_details() for customer in bank.customers]

class Transaction:
    def __init__(self, transaction_type, amount):
        self.transaction_id = str(uuid.uuid4())[:8]
        self.transaction_type = transaction_type
        self.amount = amount
        self.date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    def get_transaction_details(self):
        return f"Transaction ID: {self.transaction_id}, Type: {self.transaction_type}, Amount: {self.amount}, Date: {self.date}"

class Bank:
    def __init__(self):
        self.customers = []
        self.admins = []
    
    def create_customer_account(self, name, address, phone, account_type, initial_balance=0.0):
        customer = Customer(name, address, phone)
        if account_type == "savings":
            account = SavingsAccount(customer, initial_balance)
        elif account_type == "business":
            account = BusinessAccount(customer, initial_balance)
        else:
            raise ValueError("Invalid account type")
        
        customer.add_account(account)
        self.customers.append(customer)
        return customer, account
    
    def delete_customer_account(self, account_number):
        for customer in self.customers:
            for account in customer.accounts:
                if account.account_number == account_number:
                    customer.remove_account(account_number)
                    return "Account deleted successfully"
        return "Account not found"
    
    def view_transaction_history(self, account_number):
        for customer in self.customers:
            for account in customer.accounts:
                if account.account_number == account_number:
                    return [txn.get_transaction_details() for txn in account.transactions]
        return "Account not found"
    
    def find_customer_by_account(self, account_number):
        for customer in self.customers:
            for account in customer.accounts:
                if account.account_number == account_number:
                    return customer.get_details()
        return "Customer not found"

# Example Usage
if __name__ == "__main__":
    bank = Bank()
    
    customer, account = bank.create_customer_account("Alice", "123 Street", "1234567890", "savings", 500)
    print(account.show_account_details())
    
    account.deposit(200)
    print("Balance after deposit:", account.get_balance())
    
    try:
        account.withdraw(100)
    except ValueError as e:
        print(e)
    
    print("Balance after withdrawal:", account.get_balance())
    
    print("Transaction History:", bank.view_transaction_history(account.account_number))
